# **Insper - Eletromagnetismo e Ondulatória** #


**Aula 10 - Modelo elétrico do transformador**

**Item 5** - Calculando correntes e tensões em um transformador AC



In [1]:
#Vamos importar os módulos que precisamos
from cmath import *
from numpy import linalg
import numpy as np
import plotly.graph_objects as go

---
Na célula a seguir:

a) Declare os parâmetros de nosso transformador: $R_1=6\Omega$, $R_2=3\Omega$, $L_1=300mH$, $L_2=30mH$, frequência $f=60 Hz$ e tensão $V_1=50cos(\omega t)$.

b) Calcule:

- Frequência angular $\omega$
- Indutância Mútua $M$
- Reatâncias indutivas de $L_1$ e $L_2$, ou seja $XL1$ e $XL2$
- Reatância indutiva mútua $XM$

Obsrvação: o número complexo $j=\sqrt{-1}$ é nativo da linguagem Python. Para escrever $j$, digite "1j"

In [28]:
frequencias = np.linspace(1, 10e3, 1000)
valores_c2 = [150e-9, 0.1e-6, 0.47e-6, 1e-6, 4.7e-6]
Uf = 6  # Tensão de entrada (V1)
Rc = 10   # Resistência de carga (R_C)
R1 = 0.5  # Resistência no primário (R1)
R2 = 0.7  # Resistência no secundário (R2)
L1 = 0.01 # Indutância do primário (L1)
L2 = 0.015 # Indutância do secundário (L2)
XM = 0.02  # Reatância magnetizante (acoplamento magnético)

In [32]:
def CalcularTransformadorCapacitorParalelo(Uf, Rc, C2, freq):
    omega = 2 * np.pi * freq  # Frequência angular
    XL1 = omega * L1          # Reatância indutiva no primário
    XL2 = omega * L2          # Reatância indutiva no secundário
    XC2 = 1 / (omega * C2)    # Reatância capacitiva no secundário (em paralelo)
    
    Z = np.array([[R1 + XL1, -XM], 
                  [-XM, XL2 + (R2 * Rc) / (R2 + Rc) - XC2]])  # Impedância total
    
    V = np.array([Uf, 0])
    i = np.dot(linalg.inv(Z), V)  # Cálculo da corrente
    
    return Uf, i[1]  # Retorna V2, a corrente no secundário

In [30]:
def CalcularTransformadorCapacitorSerie(Uf, Rc, C2, freq):
    # Constantes e reatâncias
    omega = 2 * np.pi * freq  # Frequência angular
    XL1 = omega * L1          # Reatância indutiva no primário
    XL2 = omega * L2          # Reatância indutiva no secundário
    XC2 = 1 / (omega * C2)    # Reatância capacitiva no secundário (em série)
    
    # Matriz de impedância Z do transformador com capacitor em série no secundário
    Z = np.array([[R1 + XL1, -XM], 
                [-XM, XL2 + (R2 + Rc) - XC2]])  # Impedância do série
    
    # Vetor de tensões
    V = np.array([Uf, 0])
    
    # Cálculo das correntes
    i = np.dot(linalg.inv(Z), V)
    
    return i[0], i[1]  # Corrente no primário e no secundário

In [33]:
def CalcularTransformadorCapacitorParalelo(Uf, Rc, C2, freq):
    omega = 2 * np.pi * freq  # Frequência angular
    XL1 = omega * L1          # Reatância indutiva no primário
    XL2 = omega * L2          # Reatância indutiva no secundário
    XC2 = 1 / (omega * C2)    # Reatância capacitiva no secundário (em paralelo)
    
    Z = np.array([[R1 + XL1, -XM], 
                  [-XM, XL2 + (R2 * Rc) / (R2 + Rc) - XC2]])  # Impedância total
    
    V = np.array([Uf, 0])
    i = np.dot(linalg.inv(Z), V)  # Cálculo da corrente
    
    return Uf, i[1]  # Retorna V2, a corrente no secundário

# Função para simular a tensão V2 com capacitor em série
def CalcularTransformadorCapacitorSerie(Uf, Rc, C2, freq):
    omega = 2 * np.pi * freq  # Frequência angular
    XL1 = omega * L1          # Reatância indutiva no primário
    XL2 = omega * L2          # Reatância indutiva no secundário
    XC2 = 1 / (omega * C2)    # Reatância capacitiva no secundário (em série)
    
    Z = np.array([[R1 + XL1, -XM], 
                  [-XM, XL2 + (R2 + Rc - XC2)]])  # Impedância total para o circuito série
    
    V = np.array([Uf, 0])
    i = np.dot(linalg.inv(Z), V)  # Cálculo da corrente
    
    return Uf, i[1]  # Retorna V2, a corrente no secundário

# Arrays para armazenar os resultados
resultados_paralelo = {}
resultados_serie = {}

# Loop para diferentes capacitores C2
for C2 in valores_c2:
    V2_list_paralelo = []  # Lista para armazenar V2 em diferentes frequências (paralelo)
    V2_list_serie = []     # Lista para armazenar V2 em diferentes frequências (série)
    
    for freq in frequencias:
        # Simulação com C2 em paralelo
        V2_paralelo, _ = CalcularTransformadorCapacitorParalelo(Uf, Rc, C2, freq)
        V2_list_paralelo.append(np.abs(V2_paralelo))  # Armazena a magnitude da tensão V2
        
        # Simulação com C2 em série
        V2_serie, _ = CalcularTransformadorCapacitorSerie(Uf, Rc, C2, freq)
        V2_list_serie.append(np.abs(V2_serie))  # Armazena a magnitude da tensão V2
    
    resultados_paralelo[C2] = V2_list_paralelo
    resultados_serie[C2] = V2_list_serie

# Plotar os resultados usando Plotly
fig = go.Figure()

# Adicionar uma curva para cada valor de capacitor C2 (paralelo)
for C2 in valores_c2:
    fig.add_trace(go.Scatter(
        x=frequencias,
        y=resultados_paralelo[C2],
        mode='lines+markers',
        name=f'C2 = {C2} F (Paralelo)',
        line=dict(dash='solid')
    ))

# Adicionar uma curva para cada valor de capacitor C2 (série)
for C2 in valores_c2:
    fig.add_trace(go.Scatter(
        x=frequencias,
        y=resultados_serie[C2],
        mode='lines+markers',
        name=f'C2 = {C2} F (Série)',
        line=dict(dash='dash')
    ))

# Customizar o gráfico
fig.update_layout(
    title='Tensão V2 por Frequência para Diferentes Valores de Capacitor C2',
    xaxis_title='Frequência (Hz)',
    yaxis_title='Tensão V2 (V)',
    xaxis_type='log',  # Escala logarítmica para o eixo X
    yaxis_type='linear',  # Mantém o eixo Y linear
    showlegend=True
)

# Mostrar o gráfico
fig.show()

c) Observe a função que criamos na célula abaixo. Tente explicar o que ela faz.

In [6]:
def CalcularTransformador(Uf, Rc):
    Z=np.array([[R1+XL1, -XM],[-XM, XL2+R2+Rc]])
    V=np.array([Uf,0])
    i=np.dot(linalg.inv(Z),V)
    return i[0], i[1]

d) Calcule as correntes $i_1$ e $i_2$ para três condições de carga ($R_C=1M\Omega$, $R_C=100\Omega$ e $R_C=1\Omega$). Assim que calcular as correntes, calcule também $V_2$ para cada resistor de carga (analise o circuito para saber como calculá-la). Por fim, calcule as relações $N_1/N_2$, $e_1/e_2$ e $V_1/V_2$ e, o mais importante: calcule as potências no primário ($S1$) e no secundário ($S2$).

IMPORTANTE: para dar print nas relações $N_1/N_2$, $V_1/V_2$ e $e_1/e_2$, você precisará usar o **módulo dos fasores**. Para calcular o módulo de um número complexo, use "np.abs(número complexo)".

In [11]:
# Use essa célula para implementar o item d)

Rcs = [1e6, 100, 1]

for Rc in Rcs:
    i1, i2 = CalcularTransformador(Ufonte, Rc)    
    V2 = i2*Rc
    V1 = Ufonte

    e1 = XL1*i1 + XM*i2
    e2 = XM*i1 + XL2*i2
    
    S1 = V1/sqrt(2)*i1.conjugate()/sqrt(2)
    S2 = V2/sqrt(2)*i2.conjugate()/sqrt(2)
    
    print(f'Com o resistor na carga de {Rc} ohms:')
    print(f'i1 (pico): {abs(i1)} A')
    print(f'i2 (pico): {abs(i2)} A')
    print(f'V1 (pico): {abs(V1)} V')
    print(f'V2 (pico): {abs(V2)} V')
    print(f'Relação N1/N2: {abs(sqrt(L1/L2))}')
    print(f'Relação e1/e2: {abs(e1/e2)}')
    print(f'Relação V1/V2: {abs(V1/V2)}')
    print(f'S1 = {S1.real} + {S1.imag}j VA')
    print(f'S2 = {S2.real} + {S2.imag}j VA')
    print(f'Eficiência: {S2.real/S1.imag}')
    print('\n')

Com o resistor na carga de 1000000.0 ohms:
i1 (pico): 0.044209058049595525 A
i2 (pico): 1.581110888184443e-05 A
V1 (pico): 50 V
V2 (pico): 15.811108881844428 V
Relação N1/N2: 3.1622776601683795
Relação e1/e2: 3.1622776601683795
Relação V1/V2: 3.16233354495547
S1 = 0.0059883183979210376 + 1.105210228220442j VA
S2 = 0.0001249955820367699 + 0.0j VA
Eficiência: 0.00011309665694827305


Com o resistor na carga de 100 ohms:
i1 (pico): 0.06527694651469498 A
i2 (pico): 0.15261745532108567 A
V1 (pico): 50 V
V2 (pico): 15.261745532108566 V
Relação N1/N2: 3.1622776601683795
Relação e1/e2: 3.162277660168379
Relação V1/V2: 3.2761652259767424
S1 = 1.2123257541760513 + 1.092447301789855j VA
S2 = 1.1646043834341788 + 0.0j VA
Eficiência: 1.0660508580378223


Com o resistor na carga de 1 ohms:
i1 (pico): 1.0876245616424591 A
i2 (pico): 3.4372217518608763 A
V1 (pico): 50 V
V2 (pico): 3.4372217518608763 V
Relação N1/N2: 3.1622776601683795
Relação e1/e2: 3.1622776601683795
Relação V1/V2: 14.546632021320857

e) Recalcule o item **d)**, mas agora considerando que o transformador é **ideal**. Para isso, basta zerar as resistências R1 e R2 do primário e do secundário. Após zer-a-las, copie o código do item d) na célula abaixo. O que você pode concluir sobre a potência real nas três condições de carga?

In [5]:
# Use essa célula para implementar o item e)



